In [56]:
!pip install ruamel.yaml

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.4/570.4 KB 75.0 MB/s eta 0:00:00


In [18]:
import torch
import glob
import os
#torch.load("")

def go(layers):
    for f in glob.glob("/mnt/ssd-1/igor/gpt-neox/models/dense_large.logit_lens.{}/*/*.pt".format(layers)):
        ch = torch.load(f, map_location="cpu")
        if 'args' in ch:
            del ch['optimizer']
            ch['args']['num_layers'] = layers
            os.rename(f, f + ".args_orig")
            torch.save(ch, f)
        del ch

In [ ]:
for i in range(25):
    go(i)

In [ ]:
for layers in range(25):
    for f in glob.glob("/mnt/ssd-1/igor/gpt-neox/models/dense_medium.{}/*/*.pt".format(layers)):
        ch = torch.load(f)
ch

In [ ]:
import ruamel.yaml
import sys

In [ ]:
import sys
from ruamel.yaml import YAML

inp = """\
{
   # parallelism settings ( you will want to change these based on your cluster setup, ideally scheduling pipeline stages
   # across the node boundaries )
   "pipe-parallel-size": 1,
}
"""

yaml = YAML()
code = yaml.load(inp)
yaml.dump(code, sys.stdout)

In [ ]:
import yaml
file_name = '/mnt/ssd-1/igor/gpt-neox/models/dense_medium/configs/config.yml'
with open(file_name) as conf_file:
    conf = yaml.load(conf_file, Loader=yaml.FullLoader)

print(conf['num-layers'])
conf['num-layers'] = 17
conf['load'] = "mypath"
conf['save'] = "mypath"

with open('/tmp/test.yml', 'w') as f:
    yaml.dump(conf, f)

In [71]:
# Rename wandb runs

import wandb
import json
import os

api = wandb.Api()
for run in api.runs("igoro/neox"):
    if run.name != "micpie-0-0":
        print("Stopping on", run.name)
        break

    j = json.loads(run.json_config)
    if "load" in j and "value" in j["load"]:
        new_name = os.path.basename(j["load"]["value"])
    else:
        print("WARNING: Skipping", run)
        continue
    
    print("Renaming", run.name)
    run.name = new_name
    run.update()


Renaming micpie-0-0
Renaming micpie-0-0
Renaming micpie-0-0
Stopping on dense_small.in_linear_all.9


In [24]:
import torch
import glob
import os

from collections import OrderedDict

def show(x, prefix = "", path = ""):
    if type(x) is dict or type(x) is OrderedDict:
        print(prefix, path)
        for key in x.keys():
            show(x[key], prefix + "  ", str(key))
    elif type(x) is torch.Tensor:
        print(prefix, path, x.shape, x.dtype)
    elif type(x) is list:
        print(prefix, path)
        for i in range(len(x)):
            show(x[i], prefix + "  ", "[{}]".format(i))
    else:
        print(prefix, path, type(x))

def go(model, checkpoint):
    for f in glob.glob("/mnt/ssd-1/igor/gpt-neox/models/{}/{}/*.pt".format(model, checkpoint)):
        ch = torch.load(f, map_location="cpu")
        print("----", f)
        if 'optimizer' in ch:
            #print(f, ch['optimizer']["optimizer_state_dict"].keys())
            show(ch)


def load(model, checkpoint):
    for f in glob.glob("/mnt/ssd-1/igor/gpt-neox/models/{}/{}/mp_*.pt".format(model, checkpoint)):
        ch = torch.load(f, map_location="cpu")
        return ch
#go("dense_small.final_linear.7", "global_step10000")
#go("dense_small.logit_lens.7", "global_step485000")


In [ ]:
#print(c1)
#print(c2)
#c2["optimizer"]["scale_window"] = c1["optimizer"]["scale_window"]


c1 = load("dense_small.final_linear.7", "global_step10000")
c2 = load("dense_small.logit_lens.7", "global_step485000")
#c2["optimizer"]["fp32_groups_flat"] = []
torch.save(c2, "/mnt/ssd-1/igor/gpt-neox/models/dense_small.final_linear.7_exp/baseline/mp_rank_00_model_states.pt")

#shutil.copy("/mnt/ssd-1/igor/gpt-neox/models/dense_small.final_linear.7/baseline/mp_rank_00_model_states.pt", "/mnt/ssd-1/igor/gpt-neox/models/dense_small.final_linear.7_exp/baseline/")
#shutil.copy("/mnt/ssd-1/igor/gpt-neox/models/dense_small.logit_lens.7/global_step485000/mp_rank_00_model_states.pt", "/mnt/ssd-1/igor/gpt-neox/models/dense_small.final_linear.7_exp/baseline/")

In [ ]:
c1 = load("dense_small.final_linear.7", "global_step10000")

In [ ]:
print(c1.keys())

In [ ]:
cc = torch.load("/mnt/ssd-1/igor/gpt-neox/models/dense_small/global_step485000/mp_rank_00_model_states.pt")

In [ ]:
show(cc)

In [29]:
model = "dense_medium"
checkpoint = "global_step250000"
print("/mnt/ssd-1/igor/gpt-neox/models/{}/{}/*.pt".format(model, checkpoint))
for f in sorted(glob.glob("/mnt/ssd-1/igor/gpt-neox/models/{}/{}/*.pt".format(model, checkpoint))):
    ch = torch.load(f, map_location="cpu")
    print("----", f)
    show(ch)

/mnt/ssd-1/igor/gpt-neox/models/dense_medium/global_step250000/*.pt
---- /mnt/ssd-1/igor/gpt-neox/models/dense_medium/global_step250000/layer_00-model_00-model_states.pt
 
   word_embeddings.weight torch.Size([50304, 1024]) torch.float16
---- /mnt/ssd-1/igor/gpt-neox/models/dense_medium/global_step250000/layer_02-model_00-model_states.pt
 
   input_layernorm.weight torch.Size([1024]) torch.float16
   input_layernorm.bias torch.Size([1024]) torch.float16
   attention.query_key_value.weight torch.Size([3072, 1024]) torch.float16
   attention.query_key_value.bias torch.Size([3072]) torch.float16
   attention.rotary_emb.inv_freq torch.Size([64]) torch.float16
   attention.dense.weight torch.Size([1024, 1024]) torch.float16
   attention.dense.bias torch.Size([1024]) torch.float16
   post_attention_layernorm.weight torch.Size([1024]) torch.float16
   post_attention_layernorm.bias torch.Size([1024]) torch.float16
   mlp.dense_h_to_4h.weight torch.Size([4096, 1024]) torch.float16
   mlp.dense_

---- /mnt/ssd-1/igor/gpt-neox/models/dense_medium/global_step250000/layer_11-model_00-model_states.pt
 
   input_layernorm.weight torch.Size([1024]) torch.float16
   input_layernorm.bias torch.Size([1024]) torch.float16
   attention.query_key_value.weight torch.Size([3072, 1024]) torch.float16
   attention.query_key_value.bias torch.Size([3072]) torch.float16
   attention.rotary_emb.inv_freq torch.Size([64]) torch.float16
   attention.dense.weight torch.Size([1024, 1024]) torch.float16
   attention.dense.bias torch.Size([1024]) torch.float16
   post_attention_layernorm.weight torch.Size([1024]) torch.float16
   post_attention_layernorm.bias torch.Size([1024]) torch.float16
   mlp.dense_h_to_4h.weight torch.Size([4096, 1024]) torch.float16
   mlp.dense_h_to_4h.bias torch.Size([4096]) torch.float16
   mlp.dense_4h_to_h.weight torch.Size([1024, 4096]) torch.float16
   mlp.dense_4h_to_h.bias torch.Size([1024]) torch.float16
---- /mnt/ssd-1/igor/gpt-neox/models/dense_medium/global_step25000

---- /mnt/ssd-1/igor/gpt-neox/models/dense_medium/global_step250000/layer_20-model_00-model_states.pt
 
   input_layernorm.weight torch.Size([1024]) torch.float16
   input_layernorm.bias torch.Size([1024]) torch.float16
   attention.query_key_value.weight torch.Size([3072, 1024]) torch.float16
   attention.query_key_value.bias torch.Size([3072]) torch.float16
   attention.rotary_emb.inv_freq torch.Size([64]) torch.float16
   attention.dense.weight torch.Size([1024, 1024]) torch.float16
   attention.dense.bias torch.Size([1024]) torch.float16
   post_attention_layernorm.weight torch.Size([1024]) torch.float16
   post_attention_layernorm.bias torch.Size([1024]) torch.float16
   mlp.dense_h_to_4h.weight torch.Size([4096, 1024]) torch.float16
   mlp.dense_h_to_4h.bias torch.Size([4096]) torch.float16
   mlp.dense_4h_to_h.weight torch.Size([1024, 4096]) torch.float16
   mlp.dense_4h_to_h.bias torch.Size([1024]) torch.float16
---- /mnt/ssd-1/igor/gpt-neox/models/dense_medium/global_step25000